In [7]:
import os
import cv2
import time
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy import misc
from Detection import Detection
from Recognition import Recognition
import preprocess

In [8]:
#Path for reading the video
Input_video_path = "../Dataset/videos/vid1.mp4"
Output_video_path = "../Outputs/Detection_Recognition_outputs/"
Output_video_name = "Detection_Recognition.avi"
classifier_pickle_file = "../util/classifier.pkl"
O_path = Output_video_path + Output_video_name

In [9]:
#Path for the detection and recognition pb files
Detection_model_path = "../Models/Detection_mtcnn.pb"
Recognition_model_path = "../Models/Recognition_facenet.pb"

In [10]:
#Instances of detection and recognition are being created.
#Instances are created to avoid loading the graphs and sessions again and again for every frame.
detection = Detection(Detection_model_path)
recognition = Recognition(Recognition_model_path)

Detection Model Graph Initialized
Recognition Model Graph Initialized


In [11]:
with open(classifier_pickle_file, 'rb') as infile:
    model,class_names = pickle.load(infile)

In [12]:
#Initializing video capture from the Input_video_path.
cap = cv2.VideoCapture(Input_video_path)
#Variable to count frames.
frame_count = 0
#starting the processing time to calculate fps.
start = time.time()
#Ensuring the input_video_path opens without errors.
if (cap.isOpened()== False):
    print("Error opening video stream or file")

#getting the frame_width , frame_height from the given input video.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_area = frame_width * frame_height

#creating a video file to write the output frames at output_video_path(O_path).

out = cv2.VideoWriter(O_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30 , (frame_width,frame_height))

#Reading each and every frame in a while loop and processing/inferencing them through two models.
grab = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    frame_start_time = time.time()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_count = frame_count+1
    if ret != True or frame_count>1000:
        break
    if ret == True:
        
        #Detection Starts :
        
        #Inferencing the Detection model
        bbox, scores, landmarks = detection.detect(frame)
        for box, pts in zip(bbox, landmarks):
            box = box.astype('int32')
            box_w = box[3] - box[1]
            box_h = box[2] - box[0]
            box_a = box_w*box_h
            percent = box_a*100/frame_area
            
            # CROPPING THE FACES OUT OF THE IMAGE AND APPENDING THEM TO THE LIST
            print('[INFO] percentage of bounding box in total image : {:.2f}'.format(percent))
            face = np.copy(frame[box[0]:box[2] , box[1]:box[3]])
            if percent >1.0 and face.shape[0] != 0 and face.shape[1]!= 0 and face.shape[2] !=0:
                if grab == 0:
                    img = face
                grab = grab+1
#                 plt.imshow(face)
#                 plt.show()
                face = preprocess.prewhiten(face)
#                 print(face.shape)
                embedding = recognition.recognize(face = face)
                predictions = model.predict_proba(embedding)
                prediction_id = model.predict(embedding)
#                 print("PREDICTIONS :",predictions)
                best_class_indices = np.argmax(predictions, axis=1)
#                 print("BEST CLASS INDICES", best_class_indices)
                best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                print(best_class_probabilities)
#                 print("BEST CLASS PROBABILITIES", best_class_probabilities)
#                 for i in range(len(best_class_indices)):
#                     print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))
                frame = cv2.rectangle(frame, (box[1], box[0]), (box[3], box[2]), (255, 0, 0), 3)
                pts = pts.astype('int32')
                
                class_prob = str(prediction_id[0])+" : {:.2f}".format(best_class_probabilities[0])
                cv2.putText(frame, class_prob, (box[1], box[0]),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),
                            thickness=2, lineType=2)
                #Uncomment this to have landmarks
#                 for i in range(5):
#                     frame = cv2.circle(frame, (pts[i+5], pts[i]), 4, (0, 0, 255), 8)
                
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #Writing to the video output
        out.write(frame)
        
        frame_end_time = time.time()
        time_per_frame = frame_end_time - frame_start_time
        fps_frame = 1/time_per_frame
        print('[INFO] total boxes:', len(bbox))
        print('[INFO] Processing Frame:', frame_count)
        print('[INFO] Processing Speed:',fps_frame," FPS")
        print('[INFO] Time Per Frame:', time_per_frame)
        
end = time.time()
timet = end - start
fps = frame_count/timet
print("[INFO] NUMBER OF FRAMES:", frame_count)
print("[INFO] Detection took {:.5} seconds".format(end - start))
print("[INFO] Overall FPS: "+ str(fps))

# closing the writer and reader

cap.release()
out.release()

[INFO] percentage of bounding box in total image : 2.66
[0.99035985]
[INFO] percentage of bounding box in total image : 0.12
[INFO] total boxes: 2
[INFO] Processing Frame: 1
[INFO] Processing Speed: 0.6351183802717516  FPS
[INFO] Time Per Frame: 1.574509620666504
[INFO] percentage of bounding box in total image : 2.82
[0.98684974]
[INFO] percentage of bounding box in total image : 0.13
[INFO] total boxes: 2
[INFO] Processing Frame: 2
[INFO] Processing Speed: 14.13327582488678  FPS
[INFO] Time Per Frame: 0.0707550048828125
[INFO] percentage of bounding box in total image : 2.89
[0.99248635]
[INFO] percentage of bounding box in total image : 0.13
[INFO] total boxes: 2
[INFO] Processing Frame: 3
[INFO] Processing Speed: 14.462967624472851  FPS
[INFO] Time Per Frame: 0.06914210319519043
[INFO] percentage of bounding box in total image : 2.76
[0.99355086]
[INFO] total boxes: 1
[INFO] Processing Frame: 4
[INFO] Processing Speed: 14.091632649632952  FPS
[INFO] Time Per Frame: 0.07096409797668

[INFO] total boxes: 2
[INFO] Processing Frame: 34
[INFO] Processing Speed: 13.312926314453032  FPS
[INFO] Time Per Frame: 0.07511496543884277
[INFO] percentage of bounding box in total image : 3.41
[0.97858457]
[INFO] percentage of bounding box in total image : 0.15
[INFO] total boxes: 2
[INFO] Processing Frame: 35
[INFO] Processing Speed: 12.565659349712844  FPS
[INFO] Time Per Frame: 0.07958197593688965
[INFO] percentage of bounding box in total image : 3.20
[0.98742583]
[INFO] percentage of bounding box in total image : 0.15
[INFO] total boxes: 2
[INFO] Processing Frame: 36
[INFO] Processing Speed: 13.314405434575583  FPS
[INFO] Time Per Frame: 0.07510662078857422
[INFO] percentage of bounding box in total image : 3.52
[0.98381758]
[INFO] percentage of bounding box in total image : 0.14
[INFO] total boxes: 2
[INFO] Processing Frame: 37
[INFO] Processing Speed: 13.222108385689381  FPS
[INFO] Time Per Frame: 0.07563090324401855
[INFO] percentage of bounding box in total image : 3.52
[

[INFO] percentage of bounding box in total image : 5.05
[0.99257803]
[INFO] total boxes: 1
[INFO] Processing Frame: 70
[INFO] Processing Speed: 14.021254333269818  FPS
[INFO] Time Per Frame: 0.0713202953338623
[INFO] percentage of bounding box in total image : 4.70
[0.98895655]
[INFO] total boxes: 1
[INFO] Processing Frame: 71
[INFO] Processing Speed: 13.242271165892099  FPS
[INFO] Time Per Frame: 0.0755157470703125
[INFO] percentage of bounding box in total image : 4.09
[0.99250433]
[INFO] total boxes: 1
[INFO] Processing Frame: 72
[INFO] Processing Speed: 13.346562252395302  FPS
[INFO] Time Per Frame: 0.07492566108703613
[INFO] percentage of bounding box in total image : 4.98
[0.98867364]
[INFO] total boxes: 1
[INFO] Processing Frame: 73
[INFO] Processing Speed: 13.210447906922541  FPS
[INFO] Time Per Frame: 0.07569766044616699
[INFO] percentage of bounding box in total image : 4.44
[0.98717678]
[INFO] total boxes: 1
[INFO] Processing Frame: 74
[INFO] Processing Speed: 12.63382632006

[INFO] percentage of bounding box in total image : 6.73
[0.93869882]
[INFO] total boxes: 1
[INFO] Processing Frame: 119
[INFO] Processing Speed: 12.657655455464518  FPS
[INFO] Time Per Frame: 0.07900357246398926
[INFO] percentage of bounding box in total image : 5.61
[0.96455615]
[INFO] total boxes: 1
[INFO] Processing Frame: 120
[INFO] Processing Speed: 13.198060397045912  FPS
[INFO] Time Per Frame: 0.07576870918273926
[INFO] percentage of bounding box in total image : 6.12
[0.95270247]
[INFO] total boxes: 1
[INFO] Processing Frame: 121
[INFO] Processing Speed: 12.653187805069356  FPS
[INFO] Time Per Frame: 0.07903146743774414
[INFO] percentage of bounding box in total image : 5.30
[0.95434509]
[INFO] total boxes: 1
[INFO] Processing Frame: 122
[INFO] Processing Speed: 12.57462532191286  FPS
[INFO] Time Per Frame: 0.07952523231506348
[INFO] percentage of bounding box in total image : 5.84
[0.95315348]
[INFO] total boxes: 1
[INFO] Processing Frame: 123
[INFO] Processing Speed: 12.64967

[INFO] percentage of bounding box in total image : 3.66
[0.9904513]
[INFO] total boxes: 1
[INFO] Processing Frame: 158
[INFO] Processing Speed: 11.99795185720211  FPS
[INFO] Time Per Frame: 0.08334755897521973
[INFO] percentage of bounding box in total image : 3.76
[0.99015455]
[INFO] total boxes: 1
[INFO] Processing Frame: 159
[INFO] Processing Speed: 11.454964550628693  FPS
[INFO] Time Per Frame: 0.08729839324951172
[INFO] percentage of bounding box in total image : 3.70
[0.98817729]
[INFO] total boxes: 1
[INFO] Processing Frame: 160
[INFO] Processing Speed: 12.58972304016473  FPS
[INFO] Time Per Frame: 0.07942986488342285
[INFO] percentage of bounding box in total image : 3.68
[0.98954144]
[INFO] total boxes: 1
[INFO] Processing Frame: 161
[INFO] Processing Speed: 12.663426062907968  FPS
[INFO] Time Per Frame: 0.07896757125854492
[INFO] percentage of bounding box in total image : 3.88
[0.98500556]
[INFO] total boxes: 1
[INFO] Processing Frame: 162
[INFO] Processing Speed: 11.1187209

...

[INFO] total boxes: 0
[INFO] Processing Frame: 750
[INFO] Processing Speed: 19.20591979339335  FPS
[INFO] Time Per Frame: 0.05206727981567383
[INFO] total boxes: 0
[INFO] Processing Frame: 751
[INFO] Processing Speed: 18.751023765669427  FPS
[INFO] Time Per Frame: 0.053330421447753906
[INFO] total boxes: 0
[INFO] Processing Frame: 752
[INFO] Processing Speed: 19.092876424237183  FPS
[INFO] Time Per Frame: 0.05237555503845215
[INFO] total boxes: 0
[INFO] Processing Frame: 753
[INFO] Processing Speed: 19.41196290056834  FPS
[INFO] Time Per Frame: 0.051514625549316406
[INFO] total boxes: 0
[INFO] Processing Frame: 754
[INFO] Processing Speed: 18.60307012680573  FPS
[INFO] Time Per Frame: 0.053754568099975586
[INFO] total boxes: 0
[INFO] Processing Frame: 755
[INFO] Processing Speed: 18.85080449438202  FPS
[INFO] Time Per Frame: 0.053048133850097656
[INFO] total boxes: 0
[INFO] Processing Frame: 756
[INFO] Processing Speed: 18.160304814686526  FPS
[INFO] Time Per Frame: 0.05506515502929687

[INFO] total boxes: 0
[INFO] Processing Frame: 810
[INFO] Processing Speed: 18.21954832347997  FPS
[INFO] Time Per Frame: 0.0548861026763916
[INFO] total boxes: 0
[INFO] Processing Frame: 811
[INFO] Processing Speed: 17.94354652406417  FPS
[INFO] Time Per Frame: 0.055730342864990234
[INFO] percentage of bounding box in total image : 14.11
[INFO] total boxes: 1
[INFO] Processing Frame: 812
[INFO] Processing Speed: 17.99026348635816  FPS
[INFO] Time Per Frame: 0.055585622787475586
[INFO] total boxes: 0
[INFO] Processing Frame: 813
[INFO] Processing Speed: 18.962877229468546  FPS
[INFO] Time Per Frame: 0.0527346134185791
[INFO] percentage of bounding box in total image : 13.62
[0.75813774]
[INFO] total boxes: 1
[INFO] Processing Frame: 814
[INFO] Processing Speed: 12.31210720209003  FPS
[INFO] Time Per Frame: 0.08122086524963379
[INFO] total boxes: 0
[INFO] Processing Frame: 815
[INFO] Processing Speed: 16.77225141358158  FPS
[INFO] Time Per Frame: 0.05962228775024414
[INFO] total boxes: 

[INFO] total boxes: 0
[INFO] Processing Frame: 863
[INFO] Processing Speed: 19.354991139988186  FPS
[INFO] Time Per Frame: 0.051666259765625
[INFO] total boxes: 0
[INFO] Processing Frame: 864
[INFO] Processing Speed: 18.24284626927343  FPS
[INFO] Time Per Frame: 0.05481600761413574
[INFO] total boxes: 0
[INFO] Processing Frame: 865
[INFO] Processing Speed: 19.388363202684776  FPS
[INFO] Time Per Frame: 0.05157732963562012
[INFO] percentage of bounding box in total image : 7.85
[0.80339064]
[INFO] total boxes: 1
[INFO] Processing Frame: 866
[INFO] Processing Speed: 12.206050200072754  FPS
[INFO] Time Per Frame: 0.08192658424377441
[INFO] total boxes: 0
[INFO] Processing Frame: 867
[INFO] Processing Speed: 18.208474966246868  FPS
[INFO] Time Per Frame: 0.05491948127746582
[INFO] total boxes: 0
[INFO] Processing Frame: 868
[INFO] Processing Speed: 19.55094811030522  FPS
[INFO] Time Per Frame: 0.051148414611816406
[INFO] total boxes: 0
[INFO] Processing Frame: 869
[INFO] Processing Speed: 

[INFO] percentage of bounding box in total image : 11.54
[0.51420429]
[INFO] total boxes: 1
[INFO] Processing Frame: 921
[INFO] Processing Speed: 13.32413354935036  FPS
[INFO] Time Per Frame: 0.07505178451538086
[INFO] percentage of bounding box in total image : 9.39
[0.51461794]
[INFO] total boxes: 1
[INFO] Processing Frame: 922
[INFO] Processing Speed: 13.262831032901707  FPS
[INFO] Time Per Frame: 0.07539868354797363
[INFO] percentage of bounding box in total image : 9.26
[0.47977323]
[INFO] total boxes: 1
[INFO] Processing Frame: 923
[INFO] Processing Speed: 13.263208489862002  FPS
[INFO] Time Per Frame: 0.07539653778076172
[INFO] percentage of bounding box in total image : 8.11
[0.44936681]
[INFO] total boxes: 1
[INFO] Processing Frame: 924
[INFO] Processing Speed: 13.22894765892353  FPS
[INFO] Time Per Frame: 0.0755918025970459
[INFO] percentage of bounding box in total image : 8.39
[0.67314302]
[INFO] total boxes: 1
[INFO] Processing Frame: 925
[INFO] Processing Speed: 14.082217

[INFO] total boxes: 0
[INFO] Processing Frame: 967
[INFO] Processing Speed: 18.308542494216248  FPS
[INFO] Time Per Frame: 0.05461931228637695
[INFO] percentage of bounding box in total image : 7.72
[0.98601536]
[INFO] total boxes: 1
[INFO] Processing Frame: 968
[INFO] Processing Speed: 12.625421795184385  FPS
[INFO] Time Per Frame: 0.07920527458190918
[INFO] percentage of bounding box in total image : 9.25
[0.98863962]
[INFO] percentage of bounding box in total image : 0.12
[INFO] total boxes: 2
[INFO] Processing Frame: 969
[INFO] Processing Speed: 12.443716586266028  FPS
[INFO] Time Per Frame: 0.08036184310913086
[INFO] percentage of bounding box in total image : 8.26
[0.97983516]
[INFO] total boxes: 1
[INFO] Processing Frame: 970
[INFO] Processing Speed: 12.054260357750495  FPS
[INFO] Time Per Frame: 0.08295822143554688
[INFO] percentage of bounding box in total image : 8.43
[0.98881724]
[INFO] total boxes: 1
[INFO] Processing Frame: 971
[INFO] Processing Speed: 11.97308677146878  F